In [1]:
!pip install transformers -q

In [2]:
!pip install sentencepiece -q

In [3]:
!pip install pythainlp -q

In [4]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from transformers import AutoModelForSequenceClassification
import ast
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch.jit import script, trace       # hybrid frontend decorator and tracing jit
from sklearn.metrics import f1_score
from datetime import datetime
import random
import time
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

In [5]:
train_df = pd.read_csv('/content/train_finaldata.csv', index_col = 0)
test_df = pd.read_csv('/content/super3_test_title.csv', index_col = 0)
# pat_df = pd.read_csv('/kaggle/input/legal-act-classification/patterns.csv')

In [7]:
train_df

,rgno,context,pattern,question,legal_act,condition,answer,rgno_id
id,,,,,,,,
0,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['P0030'],การทำนิติกรรม สำนักงานตรวจคนเข้าเมือง,NaN,0.0,"{'นายนรากรณ์ ดีเย็น': 'P0265', 'นายนภัสกร แซ่เ..."
1,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['P0260'],การทำนิติกรรม กรมทางหสวง,NaN,0.0,"{'นายนรากรณ์ ดีเย็น': 'P0265', 'นายนภัสกร แซ่เ..."
2,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['P0164'],อสังหาริมทรัพย์,NaN,0.0,"{'นายนรากรณ์ ดีเย็น': 'P0265', 'นายนภัสกร แซ่เ..."
3,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['P0005'],การทำนิติกรรม หน่วยราชการวิสาหกิจ,NaN,0.0,"{'นายนรากรณ์ ดีเย็น': 'P0265', 'นายนภัสกร แซ่เ..."
4,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['P0265'],อสังหาริมทรัพย์,NaN,0.0,"{'นายนรากรณ์ ดีเย็น': 'P0265', 'นายนภัสกร แซ่เ..."
...,...,...,...,...,...,...,...,...
6016,1.075360e+11,กรรมการสองในเจ็ดคนลงลายมือชื่อร่วมกันและประทับ...,11004,"['P0293', 'P0118', 'P0187', 'P0031']",สัญญาเช่า,NaN,1.0,"{'นายพงศกร ชัยแก้ว': 'P0118', 'นายวิธวินท์ ศรี..."
6017,1.075360e+11,กรรมการสองในเจ็ดคนลงลายมือชื่อร่วมกันและประทับ...,11004,"['P0293', 'P0118', 'P0187', 'P0099']",ทำนา,NaN,1.0,"{'นายพงศกร ชัยแก้ว': 'P0118', 'นายวิธวินท์ ศรี..."
6018,1.075360e+11,กรรมการสองในเจ็ดคนลงลายมือชื่อร่วมกันและประทับ...,11004,"['P0293', 'P0118', 'P0187', 'P0096']",ธนาคาร,NaN,1.0,"{'นายพงศกร ชัยแก้ว': 'P0118', 'นายวิธวินท์ ศรี..."


In [9]:
test_df

,id,rgno,context,pattern,question,legal_act,condition,rgno_id
0,0,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['P0124'],การขอวีซ่า,NaN,"{'นางสาวปัทมา ยอดทอง': 'P0038', 'นายสุเมธ คุ้ม..."
1,1,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['P0052'],การค้ำประกัน,NaN,"{'นางสาวปัทมา ยอดทอง': 'P0038', 'นายสุเมธ คุ้ม..."
2,2,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['P0044'],ใบอนุญาตจำหน่ายสุราและจำหน่ายยาสูบ,NaN,"{'นางสาวปัทมา ยอดทอง': 'P0038', 'นายสุเมธ คุ้ม..."
3,3,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['P0123'],ธนาคาร,NaN,"{'นางสาวปัทมา ยอดทอง': 'P0038', 'นายสุเมธ คุ้ม..."
4,4,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['P0122'],มัคคุเทศน์,NaN,"{'นางสาวปัทมา ยอดทอง': 'P0038', 'นายสุเมธ คุ้ม..."
...,...,...,...,...,...,...,...,...
5830,5830,105562113421,กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...,20004,"['P0197', 'P0164', 'P0274']",ธุรกิจนำเที่ยว,NaN,"{'นายณัฐกานต์ โคตรยอด': 'P0066', 'นายตุลย์ เกส..."
5831,5831,105562113421,กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...,20004,"['P0197', 'P0026', 'P0274']",มัคคุเทศน์,NaN,"{'นายณัฐกานต์ โคตรยอด': 'P0066', 'นายตุลย์ เกส..."
5832,5832,105562113421,กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...,20004,"['P0066', 'P0197', 'P0274']",การทำนิติกรรม กรมที่ดิน,NaN,"{'นายณัฐกานต์ โคตรยอด': 'P0066', 'นายตุลย์ เกส..."
5833,5833,105562113421,กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...,20004,"['P0197', 'P0282', 'P0274']",การทำนิติกรรม สำนักงานส่งเริมการลงทุน,NaN,"{'นายณัฐกานต์ โคตรยอด': 'P0066', 'นายตุลย์ เกส..."


In [10]:
train_df['Text'] = train_df['context'] + ', ' + train_df['question']
train_df['Label'] = train_df['answer']
prompts = []
for index, row in train_df.iterrows():
    prompt = f'''
    คุณเป็นผู้ช่วยทางด้านนิติกรรม คุณจะต้องตอบคำถามทางด้านนิติกรรมโดยใช้ข้อมูลต้นฉบับเท่านั้น\n
    ข้อมูลต้นฉบับ : {row['context']} \n
    คำถาม : คุณ {row['question']} จะทำการ {row['legal_act']} ทำได้หรือไม่? ตอบแค่ได้ หรือ ไม่ได้ ถ้าได้ให้ตอบ 1 และถ้าไม่ได้ให้ตอบ 0'''
    prompts.append(prompt)
train_df['prompt'] = prompts
df = train_df[['Text', 'Label','prompt']]

test_df['Text'] = test_df['context'] + ', ' + test_df['question']
df_test = test_df[['Text']]

In [11]:
train_df

,rgno,context,pattern,question,legal_act,condition,answer,rgno_id,Text,Label,prompt
id,,,,,,,,,,,
0,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['P0030'],การทำนิติกรรม สำนักงานตรวจคนเข้าเมือง,NaN,0.0,"{'นายนรากรณ์ ดีเย็น': 'P0265', 'นายนภัสกร แซ่เ...",กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0.0,\n คุณเป็นผู้ช่วยทางด้านนิติกรรม คุณจะต้องต...
1,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['P0260'],การทำนิติกรรม กรมทางหสวง,NaN,0.0,"{'นายนรากรณ์ ดีเย็น': 'P0265', 'นายนภัสกร แซ่เ...",กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0.0,\n คุณเป็นผู้ช่วยทางด้านนิติกรรม คุณจะต้องต...
2,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['P0164'],อสังหาริมทรัพย์,NaN,0.0,"{'นายนรากรณ์ ดีเย็น': 'P0265', 'นายนภัสกร แซ่เ...",กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0.0,\n คุณเป็นผู้ช่วยทางด้านนิติกรรม คุณจะต้องต...
3,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['P0005'],การทำนิติกรรม หน่วยราชการวิสาหกิจ,NaN,0.0,"{'นายนรากรณ์ ดีเย็น': 'P0265', 'นายนภัสกร แซ่เ...",กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0.0,\n คุณเป็นผู้ช่วยทางด้านนิติกรรม คุณจะต้องต...
4,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['P0265'],อสังหาริมทรัพย์,NaN,0.0,"{'นายนรากรณ์ ดีเย็น': 'P0265', 'นายนภัสกร แซ่เ...",กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0.0,\n คุณเป็นผู้ช่วยทางด้านนิติกรรม คุณจะต้องต...
...,...,...,...,...,...,...,...,...,...,...,...
6016,1.075360e+11,กรรมการสองในเจ็ดคนลงลายมือชื่อร่วมกันและประทับ...,11004,"['P0293', 'P0118', 'P0187', 'P0031']",สัญญาเช่า,NaN,1.0,"{'นายพงศกร ชัยแก้ว': 'P0118', 'นายวิธวินท์ ศรี...",กรรมการสองในเจ็ดคนลงลายมือชื่อร่วมกันและประทับ...,1.0,\n คุณเป็นผู้ช่วยทางด้านนิติกรรม คุณจะต้องต...
6017,1.075360e+11,กรรมการสองในเจ็ดคนลงลายมือชื่อร่วมกันและประทับ...,11004,"['P0293', 'P0118', 'P0187', 'P0099']",ทำนา,NaN,1.0,"{'นายพงศกร ชัยแก้ว': 'P0118', 'นายวิธวินท์ ศรี...",กรรมการสองในเจ็ดคนลงลายมือชื่อร่วมกันและประทับ...,1.0,\n คุณเป็นผู้ช่วยทางด้านนิติกรรม คุณจะต้องต...
6018,1.075360e+11,กรรมการสองในเจ็ดคนลงลายมือชื่อร่วมกันและประทับ...,11004,"['P0293', 'P0118', 'P0187', 'P0096']",ธนาคาร,NaN,1.0,"{'นายพงศกร ชัยแก้ว': 'P0118', 'นายวิธวินท์ ศรี...",กรรมการสองในเจ็ดคนลงลายมือชื่อร่วมกันและประทับ...,1.0,\n คุณเป็นผู้ช่วยทางด้านนิติกรรม คุณจะต้องต...


In [12]:
train_set, val_set = train_test_split( df, test_size=0.20, random_state=42)  # test_size=0.20 for 80:20 split

train_set.reset_index(drop=True, inplace = True)
val_set.reset_index(drop=True, inplace = True)

# Print the shape of the datasets to verify the sizes
print(f"Training set shape: {train_set.shape}")
print(f"Validation set shape: {val_set.shape}")

Training set shape: (4816, 3)
Validation set shape: (1205, 3)


In [13]:
name = train_df['Text'][0]

In [14]:
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
tokenizer.add_tokens(['/content/legal_act_unique_clean_p.vocab'])
encoded = tokenizer(name, padding=True, truncation=True, return_tensors='pt')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [15]:
encoded

{'input_ids': tensor([[    5,    10,   994,    39, 24559,   154, 23597,   899,   994, 15644,
         15544, 11775,  2356,    13, 24742,  3715,   311,    14,  1599,   146,
             3,  3357,   313,   146,   711,     6]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

In [16]:
def encoder(text_seq):
  return tokenizer(text_seq, padding=True, truncation=True, return_tensors='pt')

In [17]:
model = AutoModelForSequenceClassification.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", num_labels=2)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
input_ids = []
attention_masks = []
max_len = 512

# For every tweet...
for sentence in df['Text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sentence,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df['Label'])

# Print sentence 0, now as a list of IDs.
print('Original: ', df['Text'][1684])
print('Token IDs:', input_ids[1684])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  P0074 ลงลายมือชื่อร่วมกับ P0224 หรือ P0226 รวมเป็นสองคน, ['P0226']
Token IDs: tensor([    5,    10,     3,  3357,  5736,    10,   154, 23597,   899,    10,
            3,  6445,   447,    10,    31,    10,     3,  6445,   921,    10,
        11775,  2356,    14,  1599,   146,     3,  6445,   921,   146,   711,
            6,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,    

In [19]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
#val_size = int(0.2 * len(dataset))
val_size = len(dataset)  - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

4,816 training samples
1,205 validation samples


In [20]:
train_size

4816

In [21]:
input_ids = []
attention_masks = []
max_len = 512

# For every tweet...
for sentence in df_test['Text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sentence,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
# labels = torch.tensor(df_test['Label'])

# Print sentence 0, now as a list of IDs.
print('Original: ', df_test['Text'][1684])
print('Token IDs:', input_ids[1684])

Original:  กรรมการกลุ่มที่ 1 คือ P0077 หรือ P0506 หรือ P0026 หรือ P0098 หรือ P0214 และกรรมการกลุ่มที่ 2 คือ P0103 หรือ P0133 หรือ P0147 กรรมการกลุ่มที่ 1 จำนวน 2 คน ลงลายมือชื่อร่วมกับกรรมการกลุ่มที่ 2 จำนวน 1 คน, ['P0077', 'P0103', 'P0133']
Token IDs: tensor([    5,    10,   994,  7805,    10,    59,  2168,    10,     3,  3357,
         3947,    10,    31,    10,     3,  6279,  5237,    10,    31,    10,
            3,  3357,   921,    10,    31,    10,     3,  3357,  5169,    10,
           31,    10,     3,  6445,   514,   222,   994,  7805,    10,    47,
         2168,    10,     3,  1208, 17236,    10,    31,    10,     3,  5658,
         2195,    10,    31,    10,     3,  5658,  3046,    10,   994,  7805,
           10,    59,    10,   207,    10,    47,  2246,    10,   154, 23597,
          899,   994,  7805,    10,    47,    10,   207,    10,    59,  2246,
           14,  1599,   146,     3,  3357,  3947,   146,    14,  3191,     3,
         1208, 17236,   146,    14,  3191,   

In [22]:
# Combine the training inputs into a TensorDataset.
test_dataset = TensorDataset(input_ids, attention_masks)

In [23]:
# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [24]:
# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "airesearch/wangchanberta-base-att-spm-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# if device == "cuda:0":
# # Tell pytorch to run this model on the GPU.
#     model = model.cuda()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = model.to(device)

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.sel

In [25]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [27]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(pred_flat, labels_flat)

In [28]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    # return str(datetime.timedelta(seconds=elapsed_rounded))
    return elapsed_rounded

In [32]:
print(b_input_ids.shape)
print(b_input_mask.shape)
print(b_labels)

torch.Size([16, 512])
torch.Size([16, 512])
tensor([1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0.],
       device='cuda:0', dtype=torch.float64)


In [33]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in tqdm(range(0, epochs)):

    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the device using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = output.loss
        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  F1-score: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

  0%|          | 0/4 [00:00<?, ?it/s]


======== Epoch 1 / 4 ========
Training...


OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 23.06 MiB is free. Process 305483 has 14.72 GiB memory in use. Of the allocated memory 14.56 GiB is allocated by PyTorch, and 40.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model.save_pretrained('finetune_wangchanberta')

In [ ]:
model = torch.load('bert_model')

In [ ]:
model.eval()
predictions = []
for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()

            predictions.extend(list(pred_flat))

In [ ]:
predictions

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv')
predictions_int = [int(element) for element in predictions]

for i in range(len(sample_submission['answer'])):
  sample_submission['answer'][i] = predictions_int[i]

sample_submission['answer'] = sample_submission['answer'].astype(int)

In [ ]:
sample_submission.info()

In [ ]:
sample_submission.to_csv('submission_PBookbig.csv',index=0)

In [ ]:
print(predictions_int)

In [ ]:
type(predictions_int[0])

In [ ]:
sample_submission['answer'][0]

In [ ]:
torch.save(model,'modelWangchan')

In [ ]:
model_test = torch.load('modelWangchan')